In [2]:
import pandas as pd
import numpy as np
import gzip
import os
import re

In [4]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [5]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [3]:
PATH_TO_DATA_UK = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/ukwiki_20180920/'
PATH_TO_DATA_EN = '/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/enwiki_20180920/'
PATH_TO_DATA_RL = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/'
PATH_TO_WIKIDATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_wikidata/'

### Get concurrent links for Ukrainian red links

In [3]:
uk_pagelinks = 'ukwiki-20180920-pages-links.csv.gz'

In [6]:
unpacked_pagelinks = unpack(PATH_TO_DATA_UK+uk_pagelinks)

In [7]:
df_uk_pagelinks = pd.read_csv(unpacked_pagelinks)

In [8]:
pack_and_remove(unpacked_pagelinks)

'/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/ukwiki_20180920/ukwiki-20180920-pages-links.csv.gz'

In [9]:
df_uk_pagelinks.head()

,id,link_id,link_val,is_red_link
0,13,884280,Об'єкт (філософія),False
1,13,21623,Ератосфен,False
2,13,274944,географ,False
3,13,26194,Клавдій Птолемей,False
4,13,20459,Відродження,False


In [10]:
df_uk_pagelinks.shape

(22693778, 4)

In [11]:
rlsample = pd.read_csv(PATH_TO_DATA_RL+'ukredlinks_ground_truth_3194.csv')

In [12]:
rlsample[['red link name']]

,red link name
0,Каунт Бейсі
1,аль-Каїм
2,королева
3,Prentice Hall
4,Tokar.ua
5,Monthly Notices of the Royal Astronomical Society
6,Світ (видавництво)
7,Дубович Іван Андрійович
8,помірні повітряні маси
9,Доповідь про торгівлю людьми


In [13]:
df_rl_with_all_parents = pd.merge(df_uk_pagelinks, rlsample, how='inner', left_on='link_val', right_on='red link name')

In [14]:
len(df_rl_with_all_parents['link_val'].unique())

3194

In [15]:
false_redlinks = df_rl_with_all_parents[df_rl_with_all_parents['is_red_link']==False]

In [16]:
false_redlinks['link_val'].unique()

array(['Велика Британія', 'легка атлетика', 'кінний спорт',
       'Міжнародний олімпійський комітет', 'Південна Корея',
       'Спортивне плавання', 'Боротьба (спорт)', 'академічне веслування',
       'Спортивна гімнастика', 'вітрильний спорт', '1914 у кіно',
       '1913 у кіно'], dtype=object)

In [17]:
df_rl_with_all_parents = df_rl_with_all_parents[['id', 'link_id', 'link_val']]

In [18]:
df_rl_with_all_parents.head()

,id,link_id,link_val
0,601,1782,Велика Британія
1,630,1782,Велика Британія
2,640,1782,Велика Британія
3,676,1782,Велика Британія
4,687,1782,Велика Британія


In [19]:
rl_with_concurrent = pd.merge(df_rl_with_all_parents, df_uk_pagelinks, how='left', left_on='id', right_on='id')

In [20]:
rl_with_concurrent.head()

,id,link_id_x,link_val_x,link_id_y,link_val_y,is_red_link
0,601,1782,Велика Британія,28397,відрізок,False
1,601,1782,Велика Британія,53514,пряма,False
2,601,1782,Велика Британія,256054,відстань,False
3,601,1782,Велика Британія,1545944,кінець,False
4,601,1782,Велика Британія,666498,ламана,False


In [21]:
rl_with_concurrent = rl_with_concurrent[['id', 'link_val_x', 'link_id_y', 'link_val_y']]
rl_with_concurrent.columns = ['parent_id', 'red_link_name', 'concurrent_id', 'concurrent_name']

In [22]:
rl_with_concurrent = rl_with_concurrent.drop_duplicates()

In [23]:
rl_with_concurrent.shape

(74888416, 4)

In [24]:
rl_with_concurrent[rl_with_concurrent['red_link_name']=='Latin Rite']

,parent_id,red_link_name,concurrent_id,concurrent_name
105725452,2789338,Latin Rite,309422,Сант-Феліу-де-Льобрегат
105725453,2789338,Latin Rite,2789322,Барселонська архідіоцезія
105725455,2789338,Latin Rite,-1,Latin Rite
105725456,2789338,Latin Rite,-1,Сант-Феліу-де-Льобрегатський собор
105725457,2789338,Latin Rite,1686,15 червня
105725458,2789338,Latin Rite,3659,2004
105725459,2789338,Latin Rite,-1,Агустін Кортес Соріано
105725461,2789350,Latin Rite,309415,Тарраса
105725462,2789350,Latin Rite,2789322,Барселонська архідіоцезія
105725464,2789350,Latin Rite,-1,Latin Rite


In [25]:
ukwikidata = pd.read_csv(PATH_TO_WIKIDATA+'ukwikidata.csv')

In [26]:
rl_with_concurrent['concurrent_name'] = rl_with_concurrent['concurrent_name'].str.strip()
ukwikidata['ips_site_page'] = ukwikidata['ips_site_page'].str.strip()

In [27]:
rl_with_concurrent['concurrent_name'] = rl_with_concurrent['concurrent_name'].str.lower()
ukwikidata['ips_site_page'] = ukwikidata['ips_site_page'].str.lower()

In [28]:
rl_with_concurrent['concurrent_name'] = rl_with_concurrent['concurrent_name'].str.replace('  ', ' ')
ukwikidata['ips_site_page'] = ukwikidata['ips_site_page'].str.replace('  ', ' ')

In [29]:
concurrent_with_wikidata = pd.merge(rl_with_concurrent, ukwikidata, how='left', left_on='concurrent_name', right_on='ips_site_page')

In [30]:
concurrent_with_wikidata

,parent_id,red_link_name,concurrent_id,concurrent_name,ips_item_id,ips_site_id,ips_site_page
0,601,Велика Британія,28397,відрізок,166154.0,ukwiki,відрізок
1,601,Велика Британія,53514,пряма,37105.0,ukwiki,пряма
2,601,Велика Британія,256054,відстань,126017.0,ukwiki,відстань
3,601,Велика Британія,1545944,кінець,16220058.0,ukwiki,кінець
4,601,Велика Британія,666498,ламана,633674.0,ukwiki,ламана
5,601,Велика Британія,202305,крива,161973.0,ukwiki,крива
6,601,Велика Британія,26421,границя,177239.0,ukwiki,границя
7,601,Велика Британія,54151,лінія,15878761.0,ukwiki,лінія
8,601,Велика Британія,31200,сі,188637.0,ukwiki,сі
9,601,Велика Британія,1801,сі,188637.0,ukwiki,сі


In [32]:
uk_concurrent_links_dict = {k: v['ips_item_id'].tolist() for k,v in concurrent_with_wikidata.groupby('red_link_name')}

In [33]:
uk_concurrent_links_str_dict = {k: v['concurrent_name'].tolist() for k,v in concurrent_with_wikidata.groupby('red_link_name')}

In [41]:
concurrent_with_wikidata.to_csv(PATH_TO_DATA_RL+'concurrent_links_with_wikidataID.csv')

### Get concurrent links for candidates from English Wiki

In [4]:
en_pagelinks_np = np.load(PATH_TO_DATA_EN+'results_save.npy')

In [5]:
nontransl_en_ids = pd.read_csv(PATH_TO_DATA_EN+'non_translated_en_articles.csv', encoding = 'UTF-8', sep='^')

In [6]:
nontransl_en_ids.head()

,id,title
0,309,An American in Paris
1,330,Actrius
2,332,Animalia (book)
3,344,Allan Dwan
4,359,List of Atlas Shrugged characters


In [7]:
files = [f for f in os.listdir(PATH_TO_DATA_RL+'candidates/') if re.match(r'[a-zA-Z]+_\d+.csv', f)]

In [8]:
cand_15 = pd.read_csv(PATH_TO_DATA_RL+'candidates/'+files[0], sep='^')

In [9]:
cand_15.head()

,red_link,candidate
0,Pachetra sagittigera,Phytometra viridaria
1,Pachetra sagittigera,Conistra rubiginea
2,Pachetra sagittigera,Tholera decimalis
3,Pachetra sagittigera,Pachetra sagittigera
4,Pachetra sagittigera,Hoplodrina octogenaria


In [10]:
cand_15_withid = pd.merge(cand_15, nontransl_en_ids, how='left', left_on='candidate', right_on='title')

In [11]:
cand_15_withid = cand_15_withid[['red_link', 'candidate', 'id']]

In [12]:
cand_15_withid.columns = ['red_link', 'candidate', 'candidate_id']

In [13]:
cand_15_withid.head()

,red_link,candidate,candidate_id
0,Pachetra sagittigera,Phytometra viridaria,19043714
1,Pachetra sagittigera,Conistra rubiginea,20081651
2,Pachetra sagittigera,Tholera decimalis,19058763
3,Pachetra sagittigera,Pachetra sagittigera,24455798
4,Pachetra sagittigera,Hoplodrina octogenaria,18755597


In [14]:
cand_15_withid.shape

(77323, 3)

In [15]:
en_pagelinks = pd.DataFrame(en_pagelinks_np)

In [16]:
en_pagelinks.columns = ['parent_id', 'incoming_id']

In [17]:
cand_with_parents_ids = pd.merge(cand_15_withid, en_pagelinks, how='left', left_on='candidate_id', right_on='incoming_id')

In [18]:
cand_with_parents_ids = cand_with_parents_ids[['red_link', 'candidate', 'candidate_id', 'parent_id']]

In [19]:
cand_with_parents_ids.shape

(2588659, 4)

In [20]:
cand_with_parents_ids.head()

,red_link,candidate,candidate_id,parent_id
0,Pachetra sagittigera,Phytometra viridaria,19043714,35620351
1,Pachetra sagittigera,Phytometra viridaria,19043714,32434183
2,Pachetra sagittigera,Phytometra viridaria,19043714,32434421
3,Pachetra sagittigera,Phytometra viridaria,19043714,32434699
4,Pachetra sagittigera,Phytometra viridaria,19043714,32434863


In [28]:
parent_ids_list = cand_with_parents_ids['parent_id'].drop_duplicates().values.tolist()

In [29]:
en_pagelinks_filtered = en_pagelinks[en_pagelinks['parent_id'].isin(parent_ids_list)]

In [1]:
en_pagelinks_filtered.shape

NameError: name 'en_pagelinks_filtered' is not defined

In [ ]:
concurrent_links_en = pd.merge(cand_with_parents_ids, en_pagelinks_filtered, how='left', left_on='parent_id', right_on='parent_id')